# Advance DB

## Logical Delete (Soft Delete)
Instead of physically removing data, mark it as deleted using a flag column like `is_deleted`.

In [ ]:
ALTER TABLE customer ADD COLUMN is_deleted BOOLEAN DEFAULT FALSE;

In [ ]:
UPDATE customer SET is_deleted = TRUE WHERE customer_id = 12;

## Self-Relations: Hierarchies and Recursion
A self-relation is when a row relates to another row in the **same table**, like employees and their managers.

In [ ]:
CREATE TABLE employee (
  id SERIAL PRIMARY KEY,
  name TEXT,
  manager_id INT REFERENCES employee(id)
);

### 🔸 Recursive Query (WITH RECURSIVE)
Query a chain of management hierarchy using Common Table Expressions.

In [ ]:
WITH RECURSIVE management_chain AS (
  SELECT id, name, manager_id FROM employee WHERE manager_id IS NULL
  UNION ALL
  SELECT e.id, e.name, e.manager_id
  FROM employee e
  JOIN management_chain mc ON e.manager_id = mc.id
)
SELECT * FROM management_chain;

## Indexes

### 🔸 Why Use Indexes?
- Speed up search & joins
- Reduce full table scans

In [ ]:
CREATE INDEX idx_customer_email ON customer(email);

In [ ]:
EXPLAIN ANALYZE SELECT * FROM customer WHERE email = 'x@x.com';

## Function: Reusable SQL logic that returns a result

Create a function that counts the films whose length between the 'len_from' and 'len_to' parameters

In [ ]:
CREATE FUNCTION get_film_count(len_from INT, len_to INT)
RETURNS INT
LANGUAGE plpgsql
AS
$$
DECLARE
   film_count INTEGER;
BEGIN
   SELECT COUNT(*) 
   INTO film_count
   FROM film
   WHERE length BETWEEN len_from AND len_to;
   
   RETURN film_count;
END;
$$;

SELECT get_film_count(40, 90);

## Procedure: Executes a sequence of steps, often without returning a value

In [ ]:
CREATE PROCEDURE cleanup_logs()
LANGUAGE SQL
AS $$
  DELETE FROM logs WHERE created_at < NOW() - INTERVAL '30 days';
$$;

CALL cleanup_logs();

## Trigger: Run logic automatically when data changes

In [ ]:
CREATE OR REPLACE FUNCTION update_timestamp()
RETURNS TRIGGER AS $$
BEGIN
  NEW.updated_at = NOW();
  RETURN NEW;
END;
$$ LANGUAGE plpgsql;

CREATE TRIGGER set_updated_at
BEFORE UPDATE ON customer
FOR EACH ROW
EXECUTE FUNCTION update_timestamp();

### Comparison Table
| Feature     | Trigger                         | Function                      | Procedure                     |
|-------------|----------------------------------|-------------------------------|-------------------------------|
| **When**    | Auto-run on INSERT/UPDATE/DELETE| Called in SQL (`SELECT`)     | Called with `CALL`            |
| **Returns** | VOID or `NEW`/`OLD` row         | Scalar or Table               | Usually VOID                  |
| **Use case**| Auto-maintenance                | Calculations, reuse logic     | Batch updates, cleanup jobs   |

## Transactions
Transactions should be ACID, ACID stands for:
- **Atomicity**: A transaction is treated as a single, indivisible unit. Either all operations within the transaction succeed, or none of them do.
- **Consistency**: A transaction only makes changes to the database that maintain its integrity and adherence to defined rules.
- **Isolation**: Transactions operate independently of each other, preventing interference and ensuring that each transaction sees a consistent view of the database.
- **Durability**: Once a transaction is completed successfully, its changes are permanently stored and remain persistent, even in the event of system failures.

In [ ]:
BEGIN;
UPDATE accounts SET balance = balance - 100.00
    WHERE name = 'Alice';
UPDATE accounts SET balance = balance + 100.00
    WHERE name = 'Bob';
COMMIT;